# Iteración por valor
Algoritmo desarrollado alrededor de los años 60's que usa la ecuación de optimalidad de Bellman para obtener la política óptima.

## Ecuación de optimalidad de Bellman
Propuesta en `1957` por Richard Bellman. En el contexto de los `MDP` proporciona una relación recursiva para calcular el valor de cada estado que permite calcular las políticas óptimas.

\begin{gather}
  v^*(s)=\max_{a}\sum_{s'}P(s'|s,a)\left[r+\gamma v^*(s')\right]\\
  \pi^*(s)=\arg\max_{a} v^*(s)
\end{gather}

Donde:
- $v(s)$ = es el valor del estado s.
- $a$ es una acción.
- $s'$ es el estado resultante de la acción a
- $P(s'| s, a)$ es la probabilidda de que el estado $s'$ sea el resultado de realizar la acción $a$ en el estado $s$.
- $r$ es la recompensa de de la acción $a$ en el estado $s$ que resulta en el estado $s'$
- $\gamma$ es el factor de descuento

<!-- <div align="right">
  <img src="https://drive.google.com/uc?export=view&id=1qW5SqoblzIFVBeWWxdQ5uXPbxCP_mMzD"
  alt="Value Iteration">
</div> -->

## Problema
*Obtenido de "Artificial Intelligence, A modern Approach, Stuart Russell and Peter Norvig"*

Tome en cuenta el siguiente entorno

<div align="right">
  <img src="https://drive.google.com/uc?export=view&id=11_sJaX_lpb8B7vR2YUhuUqxbKdRKd0A2"
  alt="Value Iteration">
</div>

Las acciones posibles son:
- Arriba ("U")
- Derecha ("R")
- Abajo ("D")
- Izquierda ("L")

Obtenga la política óptima tomando en cuenta un entorno determinístico $P(s', s, a) = 1$, $\gamma=1$ y una recompensa de -0.04 por moverse.

`Pista`: Si en entorno es determinístico $P(s', s, a) = 1$, la ecuación de optimalidad de Bellman se reduce a:
\begin{gather}
  v^*(s)=\max_{a}\left[r+\gamma v^*(s')\right]\\
  \pi^*(s)=\arg\max_{a} v^*(s)
\end{gather}

In [5]:
import numpy as np

In [6]:
# Initial Conditions
h, w = 3, 4
gamma = 1
r = -0.04

# Enviroment, actions and rewards
walls = [(1,1)]
states = [(i,j) for i in range(h) for j in range(w) if (i,j) not in walls]
rewards = {s:r for s in states}

rewards[(0,3)] = 1
rewards[(1,3)] = -1

end_actions = [(0,3),(1,3)]

actions = {}
for s in states:
  if s not in end_actions:
    i,j = s
    possibles = []
    if i>0 and (i-1,j) not in walls: possibles.append('U')
    if j<w-1 and (i,j+1) not in walls: possibles.append('R')
    if i<h-1 and (i+1,j) not in walls: possibles.append('D')
    if j>0 and (i,j-1) not in walls: possibles.append('L')
    actions[(i,j)]=possibles

print("States", states)
print('\nRewards:')
for pos, r in rewards.items():
  print(f'{pos}: {r}')
print()
print('Actions:')
for pos, a in actions.items():
  print(f'{pos}: {a}')

States [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]

Rewards:
(0, 0): -0.04
(0, 1): -0.04
(0, 2): -0.04
(0, 3): 1
(1, 0): -0.04
(1, 2): -0.04
(1, 3): -1
(2, 0): -0.04
(2, 1): -0.04
(2, 2): -0.04
(2, 3): -0.04

Actions:
(0, 0): ['R', 'D']
(0, 1): ['R', 'L']
(0, 2): ['R', 'D', 'L']
(1, 0): ['U', 'D']
(1, 2): ['U', 'R', 'D']
(2, 0): ['U', 'R']
(2, 1): ['R', 'L']
(2, 2): ['U', 'R', 'L']
(2, 3): ['U', 'L']


## Pseudocódigo del algoritmo de Iteración por valor

<div align="right">
  <img src="https://drive.google.com/uc?export=view&id=1JpOolQHTmkUxE9X5AclsU4JWCZ0aC_yq"
  width = 800
  alt="Value Iteration Algorithm">
</div>

`Pista`: Recuerde que para un entorno determinístico la ecuación de optimalidad de Bellman queda de la siguiente manera.
\begin{gather}
  v^*(s)=\max_{a}\left[r+\gamma v^*(s')\right]\\
  \pi^*(s)=\arg\max_{a} v^*(s)
\end{gather}


In [7]:
def next_state(action, s):
  i, j = s
  if action   == 'U': i -= 1
  elif action == 'R': j += 1
  elif action == 'D': i += 1
  elif action == 'L': j -= 1
  return i,j

In [9]:
# Value Iteration Algorithm
e = 1e-3
episodes = 0

# Initial Conditions
V = np.array([[0. for j in range(w)] for i in range(h)])
A = np.array([['' for j in range(w)] for i in range(h)])

while True:
  episodes += 1
  old_V = V.copy()
  d = 0
  for s in states:
    if s not in end_actions:
      list_of_values, list_of_actions = [], []
      for action in actions[s]:
        sp = next_state(action, s)
        list_of_values.append(rewards[s] + gamma*V[sp])
        list_of_actions.append(action)
      V[s] = np.max(list_of_values)
      A[s] = list_of_actions[np.argmax(list_of_values)]
    else:
      V[s] = rewards[s]
      A[s] = ''
    if np.abs(V[s]-old_V[s]) > d: d = np.around(np.abs(V[s]-old_V[s]),3)
  if d<=e*(1-gamma)/gamma: break
  print("Episode:", episodes)
  print(np.around(V, 3), '\n')

print("Política Óptima:")
print(A)

Episode: 1
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04  0.   -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.08]] 

Episode: 2
[[-0.08 -0.08  0.96  1.  ]
 [-0.08  0.    0.92 -1.  ]
 [-0.08 -0.08  0.88  0.84]] 

Episode: 3
[[-0.12  0.92  0.96  1.  ]
 [-0.12  0.    0.92 -1.  ]
 [-0.12  0.84  0.88  0.84]] 

Episode: 4
[[ 0.88  0.92  0.96  1.  ]
 [ 0.84  0.    0.92 -1.  ]
 [ 0.8   0.84  0.88  0.84]] 

Política Óptima:
[['R' 'R' 'R' '']
 ['U' '' 'U' '']
 ['U' 'R' 'U' 'L']]


## Resultado esperado

<div align="right">
  <img src="https://drive.google.com/uc?export=view&id=1DQRjO0Tx91LEBdDGPwQTBS5HB9oEl_W_"
  width = 600
  alt="Check results">
</div>